In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
%matplotlib qt

In [3]:
data = pd.read_csv('data-test-scan.obj', delim_whitespace=True) 
data.columns =['nothing', 'x', 'y', 'z']
data = data.drop('nothing', axis=1)

In [4]:
# Ideally, I could find the best box by converting the values from x,y,z coordinates to histogram bins that
# are ~1 mm apart. Each point would be in its own bin, and I could check all 100x100x100 boxes to find the ten
# that contain the most points. After finding one, I could delete the data points in the highest density box, 
# then find the second highest until I get all ten.

In [5]:
def find_best_box():
    maximum_value = 0
    startx = starty = startz = 0
    for xx in range(0, len(data_grid)-100):
        for yy in range(0, len(data_grid[0] - 100)):
            for zz in range(0,len(data_grid[0,0] - 100)):
#        for yy in range(200,201):               # For quick debugging
#            for zz in range(500,501):
                total = np.sum(data_grid[xx:xx+100,yy:yy+100,zz:zz+100])
                if total > maximum_value:
                    print('New Maximum Value = ' + str(total) + ' at ' + str(xx) + ',' + str(yy) + ',' + str(zz))
                    maximum_value = total
                    startx = xx
                    starty = yy
                    startz = zz
    return (startx, starty, startz, maximum_value)    

In [6]:
def find_boxes(num):
    startx = [] 
    starty = []
    startz = []
    maximum_values = []
    for _ in range(num):
        x, y, z, value = find_best_box()
        startx.append(x)
        starty.append(y)
        startz.append(z)
        print('Best box =' + str(value) + ' at ' + str(x) + ',' + str(y) + ',' + str(z))
        maximum_values.append(value)
        data_grid[x:x+100,y:y+100,z:z+100] = 0
        print(x,y,z,value)
    return (startx, starty, startz, maximum_values)

In [5]:
# Find minimum and maximum in order to find how many bins to assign in each dimension

minx = data.x.min()
maxx = data.x.max()
miny = data.y.min()
maxy = data.y.max()
maxz = data.z.max()
minz = data.z.min()

In [6]:
x_bins = (int(maxx - minx) + 1)
y_bins = (int(maxy - miny) + 1)
z_bins = (int(maxz - minz) + 1)
x_bins, y_bins, z_bins

(302, 874, 1666)

In [7]:
# Convert to 3d histogram with bins of each length.

data_grid, edges = np.histogramdd(np.array(data), bins=(x_bins, y_bins, z_bins))

In [8]:
np.sum(data_grid)
    

25002.0

In [9]:
x_vals, y_vals, z_vals, counts = find_boxes(10)

New Maximum Value = 1.0 at 0,86,1002
New Maximum Value = 2.0 at 0,87,994
New Maximum Value = 3.0 at 0,87,1002
New Maximum Value = 5.0 at 0,92,994
New Maximum Value = 6.0 at 0,92,1002
New Maximum Value = 7.0 at 0,92,1004
New Maximum Value = 8.0 at 0,92,1013
New Maximum Value = 9.0 at 0,93,1021
New Maximum Value = 10.0 at 0,93,1031
New Maximum Value = 11.0 at 0,94,1066
New Maximum Value = 12.0 at 0,96,1020
New Maximum Value = 13.0 at 0,96,1021
New Maximum Value = 14.0 at 0,96,1031
New Maximum Value = 15.0 at 0,96,1040
New Maximum Value = 16.0 at 0,96,1056
New Maximum Value = 17.0 at 0,96,1066
New Maximum Value = 18.0 at 0,97,1056
New Maximum Value = 20.0 at 0,97,1066
New Maximum Value = 21.0 at 0,101,1032
New Maximum Value = 22.0 at 0,101,1040
New Maximum Value = 23.0 at 0,101,1042
New Maximum Value = 24.0 at 0,101,1056
New Maximum Value = 25.0 at 0,101,1058
New Maximum Value = 27.0 at 0,101,1066
New Maximum Value = 28.0 at 0,101,1070
New Maximum Value = 29.0 at 0,102,1070
New Maximum Va

KeyboardInterrupt: 

In [ ]:
# Obviously (and I assumed it would, but I wanted to check) this would take way too long so I needed to come
# up with a way to approximate where the centers of the boxes would be. I can used k-means clustering over 
# a variety of values of k in order to find a number of clusters, then check which of the clusters have the most
# density. I also am going to assume that the boxes shouldn't overlap as the boxes are small in comparison to
# the number of points, so I need to make sure the boxes don't overlap.

In [7]:
def find_density(centers):
    percent_density=[]
    for center in centers:
        total_points = 0
        for ii in range(0,len(data)):
            if data.at[ii,'x'] > center[0]-50 and data.at[ii,'x'] < center[0]+50 and \
                data.at[ii,'y'] > center[1]-50 and data.at[ii, 'y'] < center[1]+50 and \
                data.at[ii,'z'] > center[2]-50 and data.at[ii, 'z'] < center[2]+50:
                total_points += 1
        percent_density.append(total_points/len(data))
    density_and_centers = pd.DataFrame(pd.concat([pd.DataFrame(centers), pd.Series(percent_density)], axis=1))
    density_and_centers.columns = ['x', 'y', 'z', 'PercentDensity']
    density_and_centers = density_and_centers.sort_values(by=['PercentDensity'], ascending=False)
    density_and_centers = density_and_centers.head(10)
    return density_and_centers

In [8]:
def find_best_centers(min, max):
    best_density = 0
    best_centers = 0
    best_number_of_clusters = 0
    for n_clusters in range(min, max):
        kmeans = KMeans(n_clusters=n_clusters)
        clusters = kmeans.fit_predict(data)
        centers = kmeans.cluster_centers_
        top_clusters = find_density(centers)
        total_coverage = top_clusters.PercentDensity.sum()
        print('For ' + str(n_clusters) + ' clusters the total density coverage is ' + str(total_coverage))
        if total_coverage > best_density:
            best_density = total_coverage
            best_centers = top_clusters
            best_number_of_clusters = n_clusters
    print('The number of clusters providing the best density coverage is ' + str(best_number_of_clusters))
    return best_centers

In [9]:
def print_cubes(df):
    df = df.reset_index()
    for ii in range(0, len(df)):
        roundx = round(df.at[ii,'x'],2)
        roundy = round(df.at[ii,'y'],2)
        roundz = round(df.at[ii,'z'],2)
        roundper = round(df.at[ii,'PercentDensity']*100,2)
        maxx = roundx + 50
        minx = roundx - 50
        maxy = roundy + 50
        miny = roundy - 50
        maxz = roundz + 50
        minz = roundz - 50
        print('Cube {0}:\n\t x: {1:0.2f} - {2:0.2f}\n\t y: {3:0.2f} - {4:0.2f}\n\t z: {5:0.2f} - {6:0.2f}\n\t Percent Coverage: {7:0.2f}'
              .format(ii+1,minx,maxx,miny,maxy,minz,maxz,roundper))


In [11]:
def calculate_overlap(df):
    df = df.reset_index()
    for ii in range(0, len(df)):
        for jj in range(0, len(df)):
            X = 0
            Y = 0
            Z = 0
            if ii == jj:
                continue
            if abs((df.at[jj,'x']) - (df.at[ii,'x'])) < 100:
                print('X range overlaps for box ' + str(ii) + ' and ' + str(jj))            
                X = 1
            if abs(df.at[jj,'y'] - df.at[ii,'y']) < 100:
                print('Y range overlaps for box ' + str(ii) + ' and ' + str(jj))
                Y = 1
            if abs(df.at[jj,'z'] - 50 - df.at[ii,'z'] + 50) < 100:
                print('Z range overlaps for box ' + str(ii) + ' and ' + str(jj))
                Z = 1
            if X and Y and Z:
                print('All three ranges overlap for box ' + str(ii) + ' and ' + str(jj))
    return True


In [12]:
best_cubes = find_best_centers(20,50)

For 20 clusters the total density coverage is 0.12710983121350292
For 21 clusters the total density coverage is 0.1329893608511319
For 22 clusters the total density coverage is 0.148468122550196
For 23 clusters the total density coverage is 0.16066714662826972
For 24 clusters the total density coverage is 0.15958723302135827
For 25 clusters the total density coverage is 0.16206703463722902
For 26 clusters the total density coverage is 0.1653467722582193
For 27 clusters the total density coverage is 0.16754659627229823
For 28 clusters the total density coverage is 0.16822654187664987
For 29 clusters the total density coverage is 0.16474682025437964
For 30 clusters the total density coverage is 0.16350691944644427
For 31 clusters the total density coverage is 0.16238700903927686
For 32 clusters the total density coverage is 0.16550675945924326
For 33 clusters the total density coverage is 0.16534677225821934
For 34 clusters the total density coverage is 0.16494680425565955
For 35 cluster

In [13]:
best_cubes


,x,y,z,PercentDensity
0,81.630876,-402.753928,754.737692,0.025918
25,118.535543,394.591002,759.961043,0.024558
24,-8.156076,-3.920865,1608.453165,0.019878
42,41.875623,407.151348,748.085827,0.019518
8,65.388694,352.182523,859.390014,0.017279
28,43.898632,-341.109595,885.274234,0.016759
3,-35.855673,-164.095782,82.246561,0.015879
44,-23.166364,169.463727,76.758835,0.015639
11,-1.243685,-201.769775,1141.162706,0.015039
43,41.404645,299.323565,960.048743,0.014999


In [24]:
calculate_overlap(best_cubes)

X range overlaps for box 0 and 1
Z range overlaps for box 0 and 1
X range overlaps for box 0 and 2
X range overlaps for box 0 and 3
Z range overlaps for box 0 and 3
X range overlaps for box 0 and 4
X range overlaps for box 0 and 5
Y range overlaps for box 0 and 5
X range overlaps for box 0 and 8
X range overlaps for box 0 and 9
X range overlaps for box 1 and 0
Z range overlaps for box 1 and 0
X range overlaps for box 1 and 3
Y range overlaps for box 1 and 3
Z range overlaps for box 1 and 3
All three ranges overlap for box 1 and 3
X range overlaps for box 1 and 4
Y range overlaps for box 1 and 4
Z range overlaps for box 1 and 4
All three ranges overlap for box 1 and 4
X range overlaps for box 1 and 5
X range overlaps for box 1 and 9
Y range overlaps for box 1 and 9
X range overlaps for box 2 and 0
X range overlaps for box 2 and 3
X range overlaps for box 2 and 4
X range overlaps for box 2 and 5
X range overlaps for box 2 and 6
X range overlaps for box 2 and 7
X range overlaps for box 2 

True

In [21]:
best_cubes2 = find_best_centers(44,45)

For 44 clusters the total density coverage is 0.1822254219662427
The number of clusters providing the best density coverage is 44


In [22]:
calculate_overlap(best_cubes2)

X range overlaps for box 0 and 1
Z range overlaps for box 0 and 1
X range overlaps for box 0 and 2
X range overlaps for box 0 and 3
Y range overlaps for box 0 and 3
X range overlaps for box 0 and 4
X range overlaps for box 0 and 7
X range overlaps for box 0 and 8
X range overlaps for box 0 and 9
X range overlaps for box 1 and 0
Z range overlaps for box 1 and 0
X range overlaps for box 1 and 2
X range overlaps for box 1 and 3
X range overlaps for box 1 and 4
Y range overlaps for box 1 and 4
X range overlaps for box 1 and 7
X range overlaps for box 1 and 8
X range overlaps for box 1 and 9
X range overlaps for box 2 and 0
X range overlaps for box 2 and 1
X range overlaps for box 2 and 3
X range overlaps for box 2 and 4
X range overlaps for box 2 and 5
X range overlaps for box 2 and 6
X range overlaps for box 2 and 7
X range overlaps for box 2 and 8
X range overlaps for box 2 and 9
X range overlaps for box 3 and 0
Y range overlaps for box 3 and 0
X range overlaps for box 3 and 1
X range ov

True

In [23]:
# 48 clusters had overlap, 44 didn't and had high coverage in the original run.

best_cubes2


,x,y,z,PercentDensity
6,84.175429,397.314142,761.089539,0.026958
11,81.857087,-403.003767,753.962528,0.026078
15,-10.281144,-4.713713,1608.883731,0.019438
14,55.137033,332.359734,897.517945,0.016919
36,44.626659,-342.584590,881.886092,0.016919
10,-34.040135,-164.285777,75.747725,0.016079
37,-24.030509,169.760545,78.436815,0.015439
38,-0.231338,-199.901583,1120.874417,0.015279
2,18.763504,-279.538535,1009.136843,0.014759
32,31.497360,265.096084,1030.882735,0.014359


In [25]:
print_cubes(best_cubes)

Cube 1:
	 x: 31.63 - 131.63
	 y: -452.75 - -352.75
	 z: 704.74 - 804.74
	 Percent Coverage: 2.59
Cube 2:
	 x: 68.54 - 168.54
	 y: 344.59 - 444.59
	 z: 709.96 - 809.96
	 Percent Coverage: 2.46
Cube 3:
	 x: -58.16 - 41.84
	 y: -53.92 - 46.08
	 z: 1558.45 - 1658.45
	 Percent Coverage: 1.99
Cube 4:
	 x: -8.12 - 91.88
	 y: 357.15 - 457.15
	 z: 698.09 - 798.09
	 Percent Coverage: 1.95
Cube 5:
	 x: 15.39 - 115.39
	 y: 302.18 - 402.18
	 z: 809.39 - 909.39
	 Percent Coverage: 1.73
Cube 6:
	 x: -6.10 - 93.90
	 y: -391.11 - -291.11
	 z: 835.27 - 935.27
	 Percent Coverage: 1.68
Cube 7:
	 x: -85.86 - 14.14
	 y: -214.10 - -114.10
	 z: 32.25 - 132.25
	 Percent Coverage: 1.59
Cube 8:
	 x: -73.17 - 26.83
	 y: 119.46 - 219.46
	 z: 26.76 - 126.76
	 Percent Coverage: 1.56
Cube 9:
	 x: -51.24 - 48.76
	 y: -251.77 - -151.77
	 z: 1091.16 - 1191.16
	 Percent Coverage: 1.50
Cube 10:
	 x: -8.60 - 91.40
	 y: 249.32 - 349.32
	 z: 910.05 - 1010.05
	 Percent Coverage: 1.50


In [27]:
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(best_cubes2.x, best_cubes2.y, best_cubes2.z, marker="x", color='black')